In [1]:
import os

# get all the .md files in this directory
files = [f for f in os.listdir('.') if os.path.isfile(f) and f.endswith('.md')]

In [30]:
import yaml

authors = []
# parse the yaml front matter between the two ---'s
for f in files:
    with open(f, 'r') as file:
        lines = file.readlines()
        start = lines.index('---\n')
        end = lines.index('---\n', start + 1)
        yaml_str = ''.join(lines[start:end])
        yaml_dict = yaml.load(yaml_str)
        yaml_dict['bio'] = ''.join(lines[end+1:])
        authors.append(yaml_dict)

C:\Users\findm\AppData\Local\Temp\ipykernel_28100\2030452099.py:11: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  yaml_dict = yaml.load(yaml_str)


In [31]:
# make a new list of the author entries, but only with the title, image, option, house, class, pronouns, and bio, if they exist
authors = [{k: v for k, v in author.items() if k in ['title', 'image', 'option', 'house', 'class', 'pronouns', 'bio']} for author in authors]

# remove \n's at front and end of bio
for author in authors:
    author['bio'] = author['bio'].strip()
    
# if the author entry only has two keys, then it's empty, so remove it
authors = [author for author in authors if len(author.keys()) > 2]

In [33]:
# for the image entry, if it exists, it's probably in the form of a URL. if it is, then download that image and save it locally
import requests
from PIL import Image
from io import BytesIO

for author in authors:
    if 'image' in author.keys():
        if author['image'].startswith('http'):
            response = requests.get(author['image'])
            img = Image.open(BytesIO(response.content))
            # make sure the format is compatible to be saved as a jpg
            if img.format == 'PNG':
                img = img.convert('RGB')
            img.save('/Users/findm/Desktop/admissions/blog_assets/profile_pics2/' + author['title'] + '.jpg')
            author['image'] = author['title'] + '.jpg'

In [36]:
# save the authors list as a yaml file. no newlines in bio
with open('authors.yaml', 'w') as file:
    yaml.dump(authors, file, default_flow_style=False, allow_unicode=True)